In [1]:
from pathlib import Path
import pandas as pd
import yaml
import numpy as np
import sys
sys.path.append('..')
from tqdm.notebook import trange

from services.adapter.src import MilvusWrapper

In [2]:
from pymilvus import CollectionSchema, DataType, FieldSchema
def create_schema(description='Museum features') -> CollectionSchema:
    img_id = FieldSchema(
        name="id",
        dtype=DataType.INT64,
        is_primary=True,
        auto_id=True
    )

    image_path = FieldSchema(
        name="image_path",
        dtype=DataType.VARCHAR,
        max_length=100,
        default_value="Unknown"
    )
    
    object_id = FieldSchema(
        name="object_id",
        dtype=DataType.INT64,
    )
    
    description = FieldSchema(
        name="description",
        dtype=DataType.VARCHAR,
        max_length=10000,
        default_value="Unknown"
    )
    
    features = FieldSchema(
        name="features",
        dtype=DataType.FLOAT_VECTOR,
        dim=1408
    )

    schema = CollectionSchema(
        fields=[img_id, image_path, object_id, description, features],
        description='Image retrieval',
        enable_dynamic_field=True
    )
    return schema

In [3]:
config = yaml.safe_load(Path('../configs/config.yaml').read_text())
milvus = MilvusWrapper(config['milvus'])
milvus.connect()
milvus.init_collection(config['collection_name'], schema=create_schema())

2024-04-13 15:50:45.305 | INFO     | services.adapter.src.wrappers.milvus_wrapper:_load_config:34 - Config has been loaded
2024-04-13 15:50:45.334 | INFO     | services.adapter.src.wrappers.milvus_wrapper:connect:43 - Milvus has been connected
2024-04-13 15:50:45.389 | INFO     | services.adapter.src.wrappers.milvus_wrapper:init_collection:71 - New collection museum_features has been added


In [9]:
data = []
for i, row in pd.read_csv('../dataset/train.csv', sep=';').fillna('').iterrows():
    image_path = Path(f'/home/borntowarn/projects/borntowarn/museum_search/storage/{row.object_id}/{row.img_name}')
    feature = np.load(image_path.with_suffix('.npy')).astype(np.float32)
    data.append({
        'image_path': f'/storage/{row.object_id}/{row.img_name}',
        'object_id': int(row.object_id),
        'description': row.description,
        'features': (feature / np.linalg.norm(feature, axis=-1, keepdims=True))[0],
    })

In [13]:
batch = 20
for i in trange(int(np.ceil(len(data) / batch))):
    milvus.insert(data[i * batch : (i + 1) * batch])

  0%|          | 0/1018 [00:00<?, ?it/s]